In [1]:
import ShapeRetrieval: ShapeRetrieval as SR
using ShapeRetrieval: LearnedTimeDiffusionBlock
import ShapeRetrieval: Mesh, heat_integrator
using LinearAlgebra
using SparseArrays
using Flux
using Zygote

### Visualization Tools

In [2]:
bunny = SR.load_obj("./meshes/gourd.obj")
bunny = SR.normalize_mesh(bunny)
println("nv: $(bunny.nv) nf: $(bunny.nf) Area: ", sum(bunny.vertex_area))
println(count(x->x != 0, bunny.cot_laplacian) / (bunny.nv ^2 ))

nv: 326 nf: 648 Area: 2.681867885523121
0.02135947909217509

In [3]:
function spectral_loss(model, x, λ, ϕ, A, y) 
    norm(model(x, λ, ϕ, A) - y)
end

spectral_loss (generic function with 1 method)

## Example Prediction using Spectral Mode

In [ ]:
m = LearnedTimeDiffusionBlock(2, :implicit)
heat_signal = zeros(bunny.nv,2)
heat_signal[1, 1] = 1.0
heat_signal[200,2] = 1.0
# λ, ϕ, A = SR.get_diffusion_inputs(bunny, :spectral)
L, A = SR.get_diffusion_inputs(bunny, :implicit)
m(heat_signal, L, M, A)

In [3]:
println(sum(bunny.face_area))
rescaled_bunny = SR.normalized_area_mesh(bunny)
println(sum(rescaled_bunny.face_area))

2.681867885523122
0.9999999999999997


In [14]:
v_sphere, f_sphere = SR.get_in_sphere(bunny, 3, 0.1)
v_reach, f_reach = SR.connected_mesh_in_sphere(bunny, 3, 0.1)

# should be false. check that connection doesn't add any additional verts/faces
println(any(.!v_sphere .& v_reach))
println(any(.!f_sphere .& f_reach))

false
false


#### Sample Training 

In [7]:
λ, ϕ, A = SR.get_diffusion_inputs(bunny, :implicit)

opt_state = Flux.setup(Adam(), m);

y_true = m(heat_signal, λ, ϕ, A)
println("t_init ", m.diffusion_time)
m = LearnedTimeDiffusionBlock(2, :implicit)
println("t_start: ", m.diffusion_time)
println("start cost ", spectral_loss(m, heat_signal, λ, ϕ, A, y_true))
@time for i=1:2000
    x, y  = heat_signal, y_true
    grad = gradient(spectral_loss, m, x, λ, ϕ, A, y)
    Flux.update!(opt_state, m, grad[1])
end
println(spectral_loss(m,heat_signal, λ, ϕ, A, y_true))
println(m)

t_init [0.6549644890868839, 0.031075904093261553]
t_start: [0.11224827247052371, 0.6408751149448856]
start cost 0.08776921174413899


  9.859870 seconds (2.46 M allocations: 22.903 GiB, 14.73% gc time, 5.22% compilation time: 30% of which was recompilation)
0.00046355064331790987


LearnedTimeDiffusionBlock(2, [0.6549644890868397, 0.030858460918697775], :

implicit)


In [6]:
m(heat_signal, SR.get_diffusion_inputs(bunny)...)
println(m.diffusion_time)

(200, 2)


[2.9274950637401447, 0.7993044171182087]


### AD with Implicit Mode

In [7]:
m = LearnedTimeDiffusionBlock(2, :implicit)
Flux.trainable(m::LearnedTimeDiffusionBlock) = (m.diffusion_time,)

function implicit_loss(model, x, L, M, A::Vector{Float64}, y)
    norm(model(x, L, M, A) - y)
end

inputs = SR.get_diffusion_inputs(bunny,:implicit)
m(heat_signal, inputs...)

0.30148325561767775

In [49]:
heat_signal = zeros(bunny.nv)
heat_signal[[1, 300]] .= 1.0
L, M, A = SR.get_diffusion_inputs(bunny, :implicit)
display(m(heat_signal, L, M, A))
data = [(heat_signal, bunny, 0.0),]

opt_state = Flux.setup(Adam(), m)

for i=1:1000
    for d in data
        x, mesh, y  = d
        grad = gradient(implicit_loss, m, x, L,M,A, y)
        Flux.update!(opt_state, m, grad[1])
    end
end

display(m(heat_signal, L, M, A))

0.3014815281221147

0.301481517244275